In [1]:
#Importing all the needed libraries

import pandas as pd 
import re
import numpy as numpy
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,r2_score
from sklearn.linear_model import LogisticRegression
import pickle

In [2]:
#download and print stop words 

nltk.download('stopwords')
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/velmurugan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#convert the given csv file into dataframe

df = pd.read_csv("/Users/velmurugan/Desktop/velu/python_works/twitter_sentiment_analysis/twitter_new.csv",encoding='latin-1')

In [4]:
df.head(2)

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...


In [5]:
#add column names to the dataframe

df = pd.read_csv("/Users/velmurugan/Desktop/velu/python_works/twitter_sentiment_analysis/twitter_new.csv",
                 encoding='latin-1',
                 names=['Target','ID','Date','Flag','User','Text'])

In [6]:
df.head(2)

,Target,ID,Date,Flag,User,Text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...


In [7]:
df.Target.value_counts()

Target
0    800000
4    800000
Name: count, dtype: int64

In [8]:
#replacing the positive target into 1 instead of 4

df.replace({"Target":{4:1}},inplace=True)
df.Target.value_counts()

Target
0    800000
1    800000
Name: count, dtype: int64

In [9]:
df.isna().sum()

Target    0
ID        0
Date      0
Flag      0
User      0
Text      0
dtype: int64

In [10]:
#using post stemmer to get the root words

portstem = PorterStemmer()

def stemming(content):
    stmmed_content = re.sub('[^ a-zA-Z]',' ',content).lower().split()
    return (" ").join([ portstem.stem(word) for word in stmmed_content if word not in stopwords.words('english')])


In [11]:
#df['Stemmed_content'] = df['Text'].apply(stemming)

In [12]:
# df_to_csv = df.to_csv("/Users/velmurugan/Desktop/velu/python_works/twitter_sentiment_analysis/stemmed_twitter.csv",index=False)

In [13]:
#convert the preprocessed csv file into dataframe

stemmed_df = pd.read_csv('/Users/velmurugan/Desktop/velu/python_works/twitter_sentiment_analysis/stemmed_twitter.csv')

In [14]:

stemmed_df.columns

Index(['Target', 'ID', 'Date', 'Flag', 'User', 'Text', 'Stemmed_content'], dtype='object')

In [15]:
stemmed_df[['Target','Stemmed_content']].head(3)

,Target,Stemmed_content
0,0,switchfoot http twitpic com zl awww bummer sho...
1,0,upset updat facebook text might cri result sch...
2,0,kenichan dive mani time ball manag save rest g...


In [16]:
#check for null values
stemmed_df.isna().sum()

Target               0
ID                   0
Date                 0
Flag                 0
User                 0
Text                 0
Stemmed_content    495
dtype: int64

In [17]:
#replacing the null values with mode

stemmed_df['Stemmed_content'].fillna(stemmed_df['Stemmed_content'].mode()[0],inplace=True)

In [18]:
stemmed_df['Stemmed_content'].isna().sum()

0

In [19]:
#defining the feature and target

X = stemmed_df['Stemmed_content'].values
y = stemmed_df['Target'].values

In [20]:
#splitting the train and test data

X_train,X_test,y_train,y_test = train_test_split(X,
                                                 y,
                                                 stratify=y,     #equally split the data with respect to y
                                                 test_size=0.1,
                                                 random_state=3)



In [21]:
X_train.shape,X_test.shape

((1440000,), (160000,))

In [22]:
X_train,y_train

(array(['sniff sunni today get done sunni',
        'petewentz hope blast yesterdamn',
        'melissa kelley ball stuck work next hr x http tinyurl com lni k',
        ..., 'listen crara', 'beach last day spring break though',
        'swear im go anoth shit hotel parti tonight im done month'],
       dtype=object),
 array([0, 1, 0, ..., 1, 1, 1]))

In [23]:
#encoding the string values

vectorizer =TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [24]:
X_train,X_test

(<1440000x497827 sparse matrix of type '<class 'numpy.float64'>'
 	with 10636330 stored elements in Compressed Sparse Row format>,
 <160000x497827 sparse matrix of type '<class 'numpy.float64'>'
 	with 1145496 stored elements in Compressed Sparse Row format>)

In [25]:
#train the logistic regression model

regg_model = LogisticRegression(max_iter=2000)
regg_model.fit(X_train,y_train)

LogisticRegression(max_iter=2000)

In [26]:
#train data prediction

y_train_pred = regg_model.predict(X_train)

train_accuracy = accuracy_score(y_train,y_train_pred)

train_accuracy

0.7992895833333333

In [27]:
#test data prediction

y_test_pred = regg_model.predict(X_test)

test_accuracy = accuracy_score(y_test,y_test_pred)

test_accuracy

0.77700625

In [28]:
#testing the random data

x = X_test[301]
print(y_test[301])

pred_n = regg_model.predict(x)

print(pred_n)

if pred_n[0] == 0:
    print("negetive tweet")
else:
    print("positive tweet")

1
[1]
positive tweet


In [29]:
#save the trained model

file_path ='/Users/velmurugan/Desktop/velu/python_works/twitter_sentiment_analysis/model.pkl'

with open(file_path,'wb') as f:
    pickle.dump(regg_model,f)

In [30]:
#function for preprocessing the raw data 

def preprocessing(twitter_post):
    stemmed_post =stemming(twitter_post)
    vectorized_X = vectorizer.transform([stemmed_post])

    return vectorized_X


In [31]:
#importing the saved model

with open(file_path,'rb') as f:
    loaded_model = pickle.load(f) 

In [32]:
# testing with own data

t = "I got good marks in the test"
pred = loaded_model.predict(preprocessing(t))

if pred[0] == 1:
    print("Positive Tweet")
elif pred[0] == 0:
    print("Negetive tweet")

Positive Tweet


In [33]:
#saving the fitted vectorizer

with open('/Users/velmurugan/Desktop/velu/python_works/twitter_sentiment_analysis/vectorizer.pkl','wb') as f:
    pickle.dump(vectorizer,f)